In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
# from OpenMax import openmax



import time

In [2]:
# head = {
#             "model" : '',
#             "attack_model": '',
#             'epsilon': '',
#             'Accuracy': '',
#             'Macro Precision': '',
#             'Weighted Precision': '',
#             'Macro Recall': '',
#             'Weighted Recall': '',
#             'Macro F1': '',
#             'Weighted F1': '',
#             # 'Macro AUC': '',
#             # 'Weighted AUC': '',
#             'TPR': '',
#             'FNR': '',
#             'TNR': '',
#             'FPR': '',
#         }
# head = pd.DataFrame([head])
# head.to_csv("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RGAN/RGAN.csv", mode='a', index=False)


In [2]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)

        np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def11/y_pred_{attack_name}{eps}_Def11.npy", all_preds)

        

In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)
        
#         accuracy = accuracy_score(all_labels, all_preds)

#         precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
#         precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
#         # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
#         # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

#         # cm = confusion_matrix(all_labels, all_preds)

#         # def calculate_class_metrics_macro(cm, class_index):
#         #     TP = cm[class_index, class_index]
#         #     FP = cm[:, class_index].sum() - TP
#         #     FN = cm[class_index, :].sum() - TP
#         #     TN = cm.sum() - (TP + FP + FN)
            
#         #     TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
#         #     TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
#         #     FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
#         #     FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
#         #     return TPR, TNR, FPR, FNR
            
#         # metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
#         # TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

#         print(f"Accuracy: {accuracy}")
        
#         print("\nmacro")
#         print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
#         print("\nweighted")
#         print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
#         print()
        
#         # print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

#         new_row = {
#             "model" : model_name,
#             "attack_model" : attack_name,
#             'epsilon': eps,
#             'Accuracy': accuracy,
#             'Macro Precision': precision_macro,
#             'Weighted Precision': precision_weighted,
#             'Macro Recall': recall_macro,
#             'Weighted Recall': recall_weighted,
#             'Macro F1': f1_macro,
#             'Weighted F1': f1_weighted,
#             # 'Macro AUC': macro_auc,
#             # 'Weighted AUC': weighted_auc,
#             # 'TPR': TPR_macro,
#             # 'FNR': FNR_macro,
#             # 'TNR': TNR_macro,
#             # 'FPR': FPR_macro,
#         }
#         new_row_df = pd.DataFrame([new_row])
#         new_row_df.to_csv("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RGAN/RGAN.csv", mode='a', index=False, header=False)



In [3]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [6]:
# class DNNModel(nn.Module):
#     def __init__(self, input_size, output_size):
#         super(DNNModel, self).__init__()
#         self.fc1 = nn.Linear(input_size, 50)
#         self.bn1 = nn.BatchNorm1d(50)
#         self.fc2 = nn.Linear(50, 30)
#         self.bn2 = nn.BatchNorm1d(30)
#         self.fc3 = nn.Linear(30, 20)
#         self.bn3 = nn.BatchNorm1d(20)
#         self.fc4 = nn.Linear(20, output_size)

#     def forward(self, x):
#         x = self.bn1(F.relu(self.fc1(x)))
#         x = self.bn2(F.relu(self.fc2(x)))
#         x = self.bn3(F.relu(self.fc3(x)))
#         logits = self.fc4(x)
#         return logits

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class Generator(nn.Module):
    def __init__(self, input_size):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, input_size)

    def forward(self, x):
        perturbation = torch.tanh(self.fc1(x))
        perturbation = self.fc2(perturbation)
        perturbation = torch.clamp(perturbation, -0.1, 0.1)
        return perturbation

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [7]:
input_size = x_train.shape[1]
output_size = len(np.unique(y_train))

In [8]:
def train_rgan(model, G, D, train_loader, val_loader, device, epochs=15, lr=0.001, patience=3, min_delta=0.001):
    optimizer_G = optim.Adam(G.parameters(), lr=lr)
    optimizer_D = optim.Adam(D.parameters(), lr=lr)
    optimizer_M = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss()

    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        G.train()
        D.train()

        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            adv_inputs = inputs + G(inputs).detach().to(device)
            adv_inputs = torch.clamp(adv_inputs, -1, 1)

            optimizer_D.zero_grad()
            real_loss = -torch.mean(D(inputs))
            fake_loss = torch.mean(D(G(inputs)))
            loss_D = real_loss + fake_loss
            loss_D.backward()
            optimizer_D.step()

            optimizer_G.zero_grad()
            fake_loss = -torch.mean(D(G(inputs)))
            fake_loss.backward()
            optimizer_G.step()

            optimizer_M.zero_grad()
            outputs = model(adv_inputs)
            loss_M = loss_function(outputs, labels)
            loss_M.backward()
            optimizer_M.step()

            train_loss += loss_M.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        correct_predictions = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_function(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_loader.dataset)

        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break


In [9]:
model = DNNModel(input_size, output_size).to(device)
G = Generator(input_size).to(device)
D = Discriminator(input_size).to(device)

train_rgan(model, G, D, train_loader, val_loader, device)


Epoch 1, Training Loss: 0.2147, Validation Loss: 0.3252, Validation Accuracy: 0.8948
Epoch 2, Training Loss: 0.1306, Validation Loss: 0.3145, Validation Accuracy: 0.8933
Epoch 3, Training Loss: 0.1179, Validation Loss: 0.3003, Validation Accuracy: 0.9050
Epoch 4, Training Loss: 0.1121, Validation Loss: 0.2810, Validation Accuracy: 0.9075
Epoch 5, Training Loss: 0.1071, Validation Loss: 0.3018, Validation Accuracy: 0.8995
Epoch 6, Training Loss: 0.1046, Validation Loss: 0.2761, Validation Accuracy: 0.9034
Epoch 7, Training Loss: 0.1014, Validation Loss: 0.2393, Validation Accuracy: 0.9191
Epoch 8, Training Loss: 0.0995, Validation Loss: 0.2518, Validation Accuracy: 0.9212
Epoch 9, Training Loss: 0.0989, Validation Loss: 0.2758, Validation Accuracy: 0.9123
Epoch 10, Training Loss: 0.0987, Validation Loss: 0.3519, Validation Accuracy: 0.8874
Early stopping triggered


In [ ]:
torch.save(model.state_dict(), "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RGAN/RGAN.pt")

In [ ]:
# model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RGAN/RGAN.pt"))

In [10]:
calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [ ]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

In [ ]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start DF")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)


print("start ZOO")
# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'ZOO', epsilon)

In [ ]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start CaFA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_CaFA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'CaFA', epsilon)

print("start SINIFGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_SINIFGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'SINIFGSM', epsilon)


print("start VNIFGSM")
# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_VNIFGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'VNIFGSM', epsilon)